In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import seaborn as sns
from sklearn.utils import resample
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import xgboost

C:\Users\ujjal\Anaconda3\lib\site-packages\distributed\utils.py:133: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [WinError 10065] A socket operation was attempted to an unreachable host
  RuntimeWarning,


In [2]:
df =pd.read_csv('train.csv')
test =pd.read_csv('test.csv')

df.head(2)

,Customer Id,YearOfObservation,Insured_Period,Residential,Building_Painted,Building_Fenced,Settlement,Building Dimension,Building_Type,Date_of_Occupancy,...,NumberOfWindows_1,NumberOfWindows_2,NumberOfWindows_3,NumberOfWindows_4,NumberOfWindows_5,NumberOfWindows_6,NumberOfWindows_7,NumberOfWindows_8,NumberOfWindows_9,NumberOfWindows_>=10
0,H14663,2013,1.0,0,0,1,1,290.0,1,1960.0,...,0,0,0,0,0,0,0,0,0,0
1,H2037,2015,1.0,0,1,0,0,490.0,1,1850.0,...,0,0,0,1,0,0,0,0,0,0


In [3]:
df.dtypes

Customer Id              object
YearOfObservation         int64
Insured_Period          float64
Residential               int64
Building_Painted          int64
Building_Fenced           int64
Settlement                int64
Building Dimension      float64
Building_Type             int64
Date_of_Occupancy       float64
Geo_Code                  int64
Claim                     int64
NumberOfWindows_   .      int64
NumberOfWindows_1         int64
NumberOfWindows_2         int64
NumberOfWindows_3         int64
NumberOfWindows_4         int64
NumberOfWindows_5         int64
NumberOfWindows_6         int64
NumberOfWindows_7         int64
NumberOfWindows_8         int64
NumberOfWindows_9         int64
NumberOfWindows_>=10      int64
dtype: object

In [4]:
class_0,class_1=df.Claim.value_counts()
minor=df[df.Claim==1]
major=df[df.Claim==0]
df_upsample=resample(minor,replace=True,n_samples=class_0)
df_upsample=pd.concat([major,df_upsample])

In [5]:
df_upsample.reset_index(drop=True,inplace=True)

In [7]:
x=df.drop(columns=['Customer Id','Claim'])
y=df['Claim']


In [9]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [10]:
from sklearn.metrics import roc_auc_score
l=[100,200,300]
for i in range(3):
    model=xgboost.XGBClassifier(max_depth=l[i],)
    model.fit(x_train,y_train)
    y_=model.predict(x_test)
    print(np.mean(y_==y_test))
    print(roc_auc_score(y_test, model.predict_proba(x_test)[:, 1]))

0.75512104283054
0.6557106440136149
0.75512104283054
0.6557106440136149
0.75512104283054
0.6557106440136149


In [22]:
model=KNeighborsClassifier(n_neighbors=5
                          )

In [23]:
#KNeighborsClassifier?

In [24]:
model.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [25]:
y_=model.predict(x_test)

In [14]:
np.mean(y_==y_test)

0.7439478584729982

In [15]:
auc?

In [30]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
def auc_csore(probs,test):
    probs = probs[:, 1]
    auc = roc_auc_score(test, probs)
    print(' AUC: %.3f' % auc)
    fpr, tpr, thresholds = roc_curve(test, probs)
    #print(fpr,tpr,thresholds)
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.plot(fpr, tpr, marker='.')

In [56]:
c=[0.0001,0.01,0.1]
for i in tol:
    model=LogisticRegression(tol=i)
    model.fit(x_train,y_train)
    y_=model.predict(x_test)
    print(np.mean(y_==y_test))
    pred=model.predict_proba(x_test)

    print(roc_auc_score(y_test, model.predict_proba(x_test)[:, 1]))

0.7811918063314711
0.7080553734151229
0.7839851024208566
0.7118462094703493
0.7830540037243948
0.711063039654866


C:\Users\ujjal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ujjal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ujjal\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [24]:
from catboost import CatBoostClassifier 
model=CatBoostClassifier(iterations=200, #leaf_estimation_iterations=10,#800
                              learning_rate=0.1,
                              depth=6,#loss_function='MultiClass',#,scale_pos_weight=200,
                             #l2_leaf_reg=5,
                             bootstrap_type='Bernoulli',
                              subsample=0.9,
                              eval_metric='AUC',
                              metric_period=20,
                                   #class_weight s=0,
                              #od_type='Iter',
                              #od_wait=45,
                              #random_seed=10,
                         plot
                              allow_writing_files=False)

In [25]:
model.fit(x_train, y_train, eval_set=(x_test, y_test))

0:	test: 0.6600628	best: 0.6600628 (0)	total: 8.99ms	remaining: 1.79s
20:	test: 0.7096119	best: 0.7096119 (20)	total: 112ms	remaining: 954ms
40:	test: 0.7122640	best: 0.7122640 (40)	total: 208ms	remaining: 806ms
60:	test: 0.7121906	best: 0.7122640 (40)	total: 298ms	remaining: 680ms
80:	test: 0.7110550	best: 0.7122640 (40)	total: 385ms	remaining: 565ms
100:	test: 0.7070387	best: 0.7122640 (40)	total: 485ms	remaining: 475ms
120:	test: 0.7053248	best: 0.7122640 (40)	total: 578ms	remaining: 377ms
140:	test: 0.7035290	best: 0.7122640 (40)	total: 673ms	remaining: 282ms
160:	test: 0.7027563	best: 0.7122640 (40)	total: 778ms	remaining: 188ms
180:	test: 0.6982525	best: 0.7122640 (40)	total: 882ms	remaining: 92.6ms
199:	test: 0.6945888	best: 0.7122640 (40)	total: 975ms	remaining: 0us

bestTest = 0.7122640374
bestIteration = 40

Shrink model to first 41 iterations.


In [27]:
test.drop(columns=['Customer Id','Claim'],inplace=True)

In [28]:
y=model.predict(test)

In [29]:
z=pd.read_csv('sub.csv')

In [24]:
xgboost.XGBClassifier?

In [66]:
z.Claim=y

In [67]:
z.to_csv('sub1.csv',index=False)

In [ ]:
0.543586032349121

In [68]:
model.predict_proba(test)

array([[0.89642098, 0.10357902],
       [0.8965798 , 0.1034202 ],
       [0.90625535, 0.09374465],
       ...,
       [0.78477947, 0.21522053],
       [0.77101561, 0.22898439],
       [0.77377487, 0.22622513]])